In [2]:
# Loading important libraries

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import sys
import sklearn.preprocessing as prepro
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
import sklearn.tree as tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer
import xgboost as xgb
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
from nltk.corpus import stopwords
import lightgbm as lgb

SEED = 50661
DATA_IMPUTE = "mean"
STOP_WORDS = stopwords.words('english')
VISUALIZE = False

ModuleNotFoundError: No module named 'lightgbm'

In [ ]:
# Initializing train/test/val file locations

trainFile = "COL774_mammography/train.csv"
testFile = "COL774_mammography/test.csv"
valFile = "COL774_mammography/val.csv"

In [ ]:
# preprocess data

trainFile = "COL774_drug_review/DrugsComTrain.csv"
testFile = "COL774_drug_review/DrugsComTest.csv"
valFile = "COL774_drug_review/DrugsComVal.csv"

vectorizer = TfidfVectorizer(stop_words = STOP_WORDS,min_df = 2)

def returnDataText(filename,isTrain = False):
    df = pd.read_csv(filename)
    df.replace(np.nan,"",inplace = True)
    
    text_vector = df["condition"] + " " + df["review"] + " "+ df["date"]
    #print(text_vector.shape)
    
    if(isTrain):
        vectorizer.fit(text_vector)
        
    X = vectorizer.transform(text_vector)
    
    #print(len(vectorizer.vocabulary_))
    #print((vectorizer.stop_words_))
    Y = np.array(df["rating"],dtype = int)
    
    return X,Y

X_train,Y_train = returnDataText(trainFile,True)
X_test,Y_test = returnDataText(testFile)
X_val,Y_val = returnDataText(valFile)

In [ ]:
"""
# preprocess data

trainFile = "COL774_mammography/train.csv"
testFile = "COL774_mammography/test.csv"
valFile = "COL774_mammography/val.csv"

def returnAgg(vector):
    if DATA_IMPUTE=="mean":
        return np.mean(vector)
    return np.median(vector)

def returnData(filename):
    df = pd.read_csv(filename)
    
    if DATA_IMPUTE!=None:
        
        ageRep = returnAgg(np.array(df[df.Age!='?']["Age"],dtype=float))
        shapeRep = returnAgg(np.array(df[df.Shape!='?']["Shape"],dtype=float))
        marginRep = returnAgg(np.array(df[df.Margin!='?']["Margin"],dtype=float))
        densityRep = returnAgg(np.array(df[df.Density!='?']["Density"],dtype=float))
        
        df.Age.replace('?',str(ageRep),inplace = True)
        df.Shape.replace('?',str(shapeRep),inplace = True)
        df.Margin.replace('?',str(marginRep),inplace = True)
        df.Density.replace('?',str(densityRep),inplace = True)
    
    df.dropna()
    df = df[(df.Age!='?') & (df.Shape!='?') & (df.Margin!='?') & (df.Density!='?')]
    
    X = np.array([df["Age"],df["Shape"],df["Margin"],df["Density"]],dtype = float).T
    Y = np.array(df["Severity"],dtype = float)
    
    #X = prepro.normalize(X)
    
    return X,Y

X_train,Y_train = returnData(trainFile)
X_test,Y_test = returnData(testFile)
X_val,Y_val = returnData(valFile)
"""

In [ ]:
# 1.a

def printAccuracies(clf):
    
    Y_train_pred = clf.predict(X_train)
    print("Training accuracy : "+str(accuracy_score(Y_train,Y_train_pred)))

    Y_test_pred = clf.predict(X_test)
    print("Testing accuracy : "+str(accuracy_score(Y_test,Y_test_pred)))

    Y_val_pred = clf.predict(X_val)
    print("Validation accuracy : "+str(accuracy_score(Y_val,Y_val_pred)))

    
clf = DecisionTreeClassifier(random_state = SEED)
clf.fit(X_train,Y_train)
printAccuracies(clf)
if(VISUALIZE): tree.plot_tree(clf)

In [ ]:
# 1.b

def doGridSearch(ourEstimator,ourParamGrid):
    grid_search = GridSearchCV(estimator = ourEstimator,param_grid = ourParamGrid)
    grid_search.fit(X_train,Y_train)

    clf = grid_search.best_estimator_
    print("Optimal parameters obtained are : "+str(grid_search.best_params_))
    printAccuracies(clf)
    
    return clf

param_grid = {'max_depth' : [1,3,5,7,9],
             'min_samples_split': [2,4,6,8,10],
             'min_samples_leaf': [1,3,7,9]}

clf = doGridSearch(DecisionTreeClassifier(random_state = SEED),param_grid)
if(VISUALIZE): tree.plot_tree(clf)

In [ ]:
#1.c 

def plotGraph(pruneDict,values,name):
    plt.plot(pruneDict['ccp_alphas'],values)
    plt.xlabel('alphas')
    plt.ylabel(name)
    plt.title(name+' vs alpha')
    #plt.savefig('1.1.c.'+name+'.png')
    plt.figure()

clf = DecisionTreeClassifier(random_state = SEED)
pruneDict = clf.cost_complexity_pruning_path(X = X_train,y = Y_train)

plotGraph(pruneDict,pruneDict['impurities'],'impurities')

In [ ]:
numNodesList = []
depthList = []
trainList = []
testList = []
valList = []

for thisAlpha in pruneDict["ccp_alphas"]:
    clf = DecisionTreeClassifier(ccp_alpha = thisAlpha,random_state = SEED)
    clf.fit(X_train,Y_train)
    
    numNodesList.append(clf.tree_.node_count)
    depthList.append(clf.tree_.max_depth)
    
    Y_train_pred = clf.predict(X_train)
    trainList.append(accuracy_score(Y_train,Y_train_pred))

    Y_test_pred = clf.predict(X_test)
    testList.append(accuracy_score(Y_test,Y_test_pred))

    Y_val_pred = clf.predict(X_val)
    valList.append(accuracy_score(Y_val,Y_val_pred))

plotGraph(pruneDict,numNodesList,'num_nodes')
plotGraph(pruneDict,depthList,'max_depth')
plotGraph(pruneDict,trainList,'train_accu')
plotGraph(pruneDict,testList,'test_accu')
plotGraph(pruneDict,valList,'val_accu')

In [ ]:
# 1.d

param_grid = {'n_estimators' : [50,100,150,200,250,300,350,400,450],
              'max_features' : [0.4,0.5,0.6,0.7,0.8],
              'min_samples_split': [2,4,6,8,10]}

best_clf = doGridSearch(RandomForestClassifier(random_state = SEED,oob_score = True),param_grid)
print('OOB score : '+str(best_clf.oob_score_))

In [ ]:
# 1.e
param_grid = {'n_estimators' : [50,100,150,200,250,300,350,400,450],
              'subsample' : [0.4,0.5,0.6,0.7,0.8],
              'max_depth' : [40,50,60,70]}

clf = doGridSearch(xgb.XGBClassifier(),param_grid)